# 智谱 AI 图片理解 (Vision) 测试

本 Notebook 测试智谱 AI 的视觉理解能力，包括三个模型：
- **GLM-4.6V-Flash**: 支持 50 张图片，支持 Base64，支持思考模式
- **GLM-4.1V-Thinking-Flash**: 支持 50 张图片，支持 Base64，内置思考模式（强制思考）
- **GLM-4V-Flash**: 仅支持 1 张图片，不支持 Base64，不支持思考模式

## 1. 环境配置

In [ ]:
import requests
import base64
import json
from IPython.display import Image as IPImage, display

# API 配置
API_KEY = "f7ec61a7856a4f4389e037eb588c69df.0pZgcFlTCk9GHCXM"
API_URL = "https://open.bigmodel.cn/api/paas/v4/chat/completions"

# 测试图片 URL
TEST_IMAGE_URL = "https://cdn.bigmodel.cn/static/logo/register.png"
TEST_IMAGE_URL_2 = "https://cdn.bigmodel.cn/static/logo/login.png"

headers = {
    "Authorization": f"Bearer {API_KEY}",
    "Content-Type": "application/json"
}

def vision_chat(model, content, max_tokens=None):
    """通用视觉对话函数"""
    data = {
        "model": model,
        "messages": [{"role": "user", "content": content}]
    }
    if max_tokens:
        data["max_tokens"] = max_tokens
    
    response = requests.post(API_URL, headers=headers, json=data)
    return response.json()

print("✅ 环境配置完成")

### 测试图片展示

In [ ]:
# 显示测试图片
print("测试图片 1:")
display(IPImage(url=TEST_IMAGE_URL, width=400))

print("\n测试图片 2:")
display(IPImage(url=TEST_IMAGE_URL_2, width=400))

## 2. GLM-4.6V-Flash 单张图片理解

In [ ]:
# 单张图片描述
content = [
    {"type": "text", "text": "请详细描述这张图片的内容"},
    {"type": "image_url", "image_url": {"url": TEST_IMAGE_URL}}
]

result = vision_chat("glm-4.6v-flash", content)
print("模型输出:")
print(result["choices"][0]["message"]["content"])

In [ ]:
# 视觉问答
content = [
    {"type": "text", "text": "这张图片中的 logo 是什么颜色？有什么文字？"},
    {"type": "image_url", "image_url": {"url": TEST_IMAGE_URL}}
]

result = vision_chat("glm-4.6v-flash", content)
print("问答结果:")
print(result["choices"][0]["message"]["content"])

## 3. GLM-4.6V-Flash 多张图片对比分析

In [ ]:
# 多张图片对比（GLM-4.6V-Flash 支持最多 50 张）
content = [
    {"type": "text", "text": "请比较这两张图片的异同点"},
    {"type": "image_url", "image_url": {"url": TEST_IMAGE_URL}},
    {"type": "image_url", "image_url": {"url": TEST_IMAGE_URL_2}}
]

result = vision_chat("glm-4.6v-flash", content)
print("对比分析结果:")
print(result["choices"][0]["message"]["content"])

## 4. GLM-4.6V-Flash Base64 图片测试

In [ ]:
# 下载图片并转为 Base64
import urllib.request

# 下载测试图片
temp_image_path = "/tmp/test_image.png"
urllib.request.urlretrieve(TEST_IMAGE_URL, temp_image_path)

# 转为 Base64
with open(temp_image_path, "rb") as f:
    base64_image = base64.b64encode(f.read()).decode('utf-8')

print(f"✅ Base64 编码完成，长度: {len(base64_image)} 字符")

In [ ]:
# 使用 Base64 调用
content = [
    {"type": "text", "text": "描述这张图片"},
    {
        "type": "image_url",
        "image_url": {
            "url": f"data:image/png;base64,{base64_image}"
        }
    }
]

result = vision_chat("glm-4.6v-flash", content)
print("Base64 调用结果:")
print(result["choices"][0]["message"]["content"])

## 5. GLM-4.1V-Thinking-Flash 图片推理测试

In [ ]:
# 使用思考模型进行深度分析
content = [
    {
        "type": "text", 
        "text": "请仔细观察这张图片，分析它的设计理念、色彩搭配和可能的用途，并解释你的推理过程"
    },
    {"type": "image_url", "image_url": {"url": TEST_IMAGE_URL}}
]

result = vision_chat("glm-4.1v-thinking-flash", content)
message = result["choices"][0]["message"]

print("=== 推理过程 ===")
print(message.get("reasoning_content", "无推理过程"))
print("\n=== 最终答案 ===")
print(message["content"])

## 6. GLM-4V-Flash 轻量图片理解

**⚠️ 注意：GLM-4V-Flash 仅支持 1 张图片，且不支持 Base64 编码**

In [ ]:
# GLM-4V-Flash 单张图片（正常工作）
content = [
    {"type": "text", "text": "这是什么图片？"},
    {"type": "image_url", "image_url": {"url": TEST_IMAGE_URL}}
]

result = vision_chat("glm-4v-flash", content, max_tokens=1000)
print("GLM-4V-Flash 输出:")
print(result["choices"][0]["message"]["content"])

In [ ]:
# ⚠️ 测试：GLM-4V-Flash 传入多张图片会报错
content = [
    {"type": "text", "text": "比较这两张图"},
    {"type": "image_url", "image_url": {"url": TEST_IMAGE_URL}},
    {"type": "image_url", "image_url": {"url": TEST_IMAGE_URL_2}}
]

result = vision_chat("glm-4v-flash", content)
print("错误信息:")
print(json.dumps(result, indent=2, ensure_ascii=False))

## 7. 三个模型效果对比

**模型能力对比：**
| 模型 | 思考模式 | 思维链 | 图片数量 | Base64 |
|:---|:---:|:---:|:---:|:---:|
| GLM-4.6V-Flash | ✅ 支持 | ✅ 支持 | 50张 | ✅ |
| GLM-4.1V-Thinking-Flash | ✅ 内置 | ✅ 支持 | 50张 | ✅ |
| GLM-4V-Flash | ❌ 不支持 | ❌ 不支持 | 1张 | ❌ |

In [ ]:
# 使用相同图片和问题对比三个模型
question = "请描述这张图片，并分析它的设计特点"
content = [
    {"type": "text", "text": question},
    {"type": "image_url", "image_url": {"url": TEST_IMAGE_URL}}
]

models = {
    "GLM-4.6V-Flash": "glm-4.6v-flash",
    "GLM-4.1V-Thinking-Flash": "glm-4.1v-thinking-flash",
    "GLM-4V-Flash": "glm-4v-flash"
}

for name, model_code in models.items():
    print(f"\n{'='*50}")
    print(f"🤖 {name}")
    print('='*50)
    
    result = vision_chat(model_code, content, max_tokens=1000 if "4v-flash" in model_code else None)
    message = result["choices"][0]["message"]
    
    if "reasoning_content" in message and message["reasoning_content"]:
        print(f"\n[推理过程]: {message['reasoning_content'][:200]}...")
    print(f"\n[输出]: {message['content']}")

## 8. OCR 文字识别测试

In [ ]:
# 测试 OCR 能力 - 提取图片中的文字
content = [
    {
        "type": "text", 
        "text": "请提取这张图片中的所有文字，并说明文字的位置和样式"
    },
    {"type": "image_url", "image_url": {"url": TEST_IMAGE_URL}}
]

result = vision_chat("glm-4.6v-flash", content)
print("OCR 识别结果:")
print(result["choices"][0]["message"]["content"])

## 9. 错误处理测试

In [ ]:
# 测试 1: 无效的图片 URL
content = [
    {"type": "text", "text": "描述这张图片"},
    {"type": "image_url", "image_url": {"url": "https://invalid-url.com/not-exist.jpg"}}
]

result = vision_chat("glm-4.6v-flash", content)
print("无效 URL 错误:")
print(json.dumps(result, indent=2, ensure_ascii=False))

In [ ]:
# 测试 2: GLM-4V-Flash 超过数量限制
content = [
    {"type": "text", "text": "分析这些图片"},
    {"type": "image_url", "image_url": {"url": TEST_IMAGE_URL}},
    {"type": "image_url", "image_url": {"url": TEST_IMAGE_URL_2}}
]

result = vision_chat("glm-4v-flash", content)
print("\n数量限制错误:")
if "error" in result:
    print(f"错误码: {result['error'].get('code', 'N/A')}")
    print(f"错误信息: {result['error'].get('message', 'N/A')}")
else:
    print(json.dumps(result, indent=2, ensure_ascii=False))

In [ ]:
# 测试 3: GLM-4V-Flash 不支持 Base64
content = [
    {"type": "text", "text": "描述这张图片"},
    {
        "type": "image_url",
        "image_url": {"url": f"data:image/png;base64,{base64_image}"}
    }
]

result = vision_chat("glm-4v-flash", content)
print("\nBase64 不支持错误:")
if "error" in result:
    print(f"错误码: {result['error'].get('code', 'N/A')}")
    print(f"错误信息: {result['error'].get('message', 'N/A')}")
else:
    print(json.dumps(result, indent=2, ensure_ascii=False))

In [ ]:
# 测试 4: 不支持的图片格式
content = [
    {"type": "text", "text": "描述这张图片"},
    {"type": "image_url", "image_url": {"url": "https://example.com/image.gif"}}
]

result = vision_chat("glm-4.6v-flash", content)
print("\n格式不支持错误:")
if "error" in result:
    print(f"错误码: {result['error'].get('code', 'N/A')}")
    print(f"错误信息: {result['error'].get('message', 'N/A')}")
else:
    print(json.dumps(result, indent=2, ensure_ascii=False))

## 总结

### 模型特性对比

| 特性 | GLM-4.6V-Flash | GLM-4.1V-Thinking-Flash | GLM-4V-Flash |
|:---|:---:|:---:|:---:|
| 上下文长度 | 128K | 64K | 16K |
| 最大输出 | 32K | 16K | **1K** |
| 思考模型 | ✅ | ✅ | ❌ |
| 图片数量 | **50张** | **50张** | **1张** ⚠️ |
| Base64 支持 | ✅ | ✅ | ❌ ⚠️ |

### 使用建议

1. **GLM-4.6V-Flash**: 推荐作为主力视觉模型，支持多图、Base64、长输出
2. **GLM-4.1V-Thinking-Flash**: 需要深度推理和分析时使用，带思考过程
3. **GLM-4V-Flash**: 仅适合简单的单图快速识别，注意单张和仅 URL 的限制